In [ ]:
#r "nuget: System.ClientModel, 1.8.1"

In [ ]:

#r "nuget: Microsoft.Extensions.AI.Abstractions, 10.2.0"

In [ ]:
#r "nuget: Microsoft.Extensions.AI, 10.2.0"

In [ ]:

#r "nuget: Azure.AI.Agents.Persistent, 1.2.0-beta.8"
#r "nuget: Azure.Identity, 1.18.0-beta.2"
#r "nuget: System.Linq.Async, 7.0.0"

In [ ]:
#r "nuget: Azure.AI.Projects, 1.2.0-beta.5"

In [ ]:
#r "nuget: Microsoft.Agents.AI.AzureAI, 1.0.0-preview.260108.1"

In [ ]:
#r "nuget: Microsoft.Agents.AI, 1.0.0-preview.260108.1"

In [ ]:
#r "nuget: DotNetEnv, 3.1.1"

In [ ]:
using System.ClientModel;
using Azure.AI.Projects;
using Azure.Identity;
using Microsoft.Agents.AI;
using Microsoft.Extensions.AI;
using OpenAI;
using OpenAI.Files;
using OpenAI.VectorStores;
using DotNetEnv;

In [ ]:
Env.Load("../../../.env");

In [ ]:
var endpoint = Environment.GetEnvironmentVariable("AZURE_AI_PROJECT_ENDPOINT");
var deploymentName = Environment.GetEnvironmentVariable("AZURE_AI_MODEL_DEPLOYMENT_NAME");

In [ ]:
AIProjectClient aiProjectClient = new(
    new Uri(endpoint),
    new AzureCliCredential());

In [ ]:
OpenAIClient openAIClient = aiProjectClient.GetProjectOpenAIClient();

In [ ]:
OpenAIFileClient fileClient = openAIClient.GetOpenAIFileClient();
ClientResult<OpenAIFile> uploadResult = await fileClient.UploadFileAsync(
    filePath: "./document.md",
    purpose: FileUploadPurpose.Assistants);

In [ ]:
#pragma warning disable OPENAI001
VectorStoreClient vectorStoreClient = openAIClient.GetVectorStoreClient();
ClientResult<VectorStore> vectorStoreCreate = await vectorStoreClient.CreateVectorStoreAsync(options: new VectorStoreCreationOptions()
{
    Name = "contoso-outdoors-knowledge-base",
    FileIds = { uploadResult.Value.Id }
});
#pragma warning restore OPENAI001

In [ ]:
var fileSearchTool = new HostedFileSearchTool() { Inputs = [new HostedVectorStoreContent(vectorStoreCreate.Value.Id)] };


In [ ]:
AIAgent agent = await aiProjectClient
    .CreateAIAgentAsync(
        model: deploymentName,
        name: "dotNETRAGAgent",
        instructions: @"You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s). 
                If a user's question cannot be answered using the retrieved context, you must clearly respond: 
                'I'm sorry, but the uploaded document does not contain the necessary information to answer that question.' 
                Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations. 
                For questions that do have relevant content in the document, respond accurately and cite the document explicitly.",
        tools: [fileSearchTool]);

In [ ]:
AgentThread thread = await agent.GetNewThreadAsync();

In [ ]:
await agent.RunAsync("Can you explain Contoso's travel insurance coverage?", thread)